<a href="https://colab.research.google.com/github/YorkJong/news-digest/blob/main/notebooks/news_subscrip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install

In [ ]:
import os

fns = ['clip.py', 'line.py', 'op.py', 'hashtag.py']

for fn in fns:
    if os.path.exists(fn):
        os.remove(fn)
for fn in fns:
    url = f'https://raw.githubusercontent.com/YorkJong/news-digest/main/api/{fn}'
    !wget $url

#-------------------------------------------------------------------------------
# Initializing
#-------------------------------------------------------------------------------

import clip
import line
import hashtag

from google.colab import drive
drive.mount('/content/drive')

### Execute Actions Step by Step

In [2]:
#@title Step 1. Pick a **period** of the news { run: "auto", display-mode: "form" }
period = "Recent 7 Days" #@param ["Today", "Yesterday", "Recent 2 Days", "Recent 7 Days"]


def news_today():
    return clip.get_latest_journal()

def news_yesterday():
    fns = clip.get_recent_journal_filenames(2)
    return clip.get_journal(fns[-2])

def news_recent2days():
    return clip.merge_recent_journals(days=2)

def news_recent7days():
    return clip.merge_recent_journals(days=7)

period2func = {
    'Today': news_today,
    'Yesterday': news_yesterday,
    'Recent 2 Days': news_recent2days,
    'Recent 7 Days': news_recent7days,
}

content = period2func[period]()

import ipywidgets as widgets
output = widgets.Output()
with output:
    print(content)

In [3]:
#@title Step 2. Pick a publication **frequency** { run: "auto", display-mode: "form" }
frequency = "Weekly" #@param ["Daily", "Weekly", "Any"]

import yaml

subscriptions = (
    (('Tesla & SpaceX; Vehicle'), ('GroupA', 'GroupB')),
    (('Taiwan',), ('GroupA', 'GroupC')),
    (('Crypto',), ('GroupC',)),
    (('IT', 'Science', '#AI', '#Robot'), ('GroupB',)),
)

path = '/content/drive/My Drive/news_subscrip'
fn = f'{path}/subscriptions_{frequency}.yml'
if os.path.exists(fn):
    with open(fn, 'r') as f:
        subscriptions = yaml.safe_load(f)

In [8]:
#@title Step 3. Line Notify { display-mode: "form" }
mock_mode = True #@param {type:"boolean"}
show_headers = False #@param {type:"boolean"}

import time
import ipywidgets as widgets
import yaml

tok_tbl = {
    'GroupA': 'LINE_NOTIFY_ACCESS_TOKEN_OF_GROUP_A',
    'GroupB': 'LINE_NOTIFY_ACCESS_TOKEN_OF_GROUP_B',
    'GroupC': 'LINE_NOTIFY_ACCESS_TOKEN_OF_GROUP_C',
}
path = '/content/drive/My Drive/news_subscrip'
fn = f'{path}/access_tokens.yml'
if os.path.exists(fn):
    with open(fn, 'r') as f:
        tok_tbl = yaml.safe_load(f)

def create_outputs():
    topices = [str(t) for t, _ in subscriptions]
    tab = widgets.Tab()
    outputs = [widgets.Output() for name in topices]
    tab.children = outputs
    for i, t in enumerate(topices):
        tab.set_title(i, t)
    display(tab)
    return outputs

def notify():
    if mock_mode:
        outputs = create_outputs()
    for i, subscription in enumerate(subscriptions):
        topices, clients = subscription
        headers = [topice for topice in topices if not topice.startswith('#')]
        tags = [topice for topice in topices if topice.startswith('#')]
        categories = headers 
        if not categories and tags:
            categories = clip.get_categories(content)
        if tags:
            lines = clip.get_lines_of_categories(categories, content, True, True) 
            lines = hashtag.get_lines_with_any_hashtags(lines, tags)
            with_headers = True if headers and show_headers else False
            lines = clip.get_lines_of_categories(categories, '\n'.join(lines), False, with_headers)
        else:
            lines = clip.get_lines_of_categories(categories, content, False, show_headers) 
        if not lines: 
            continue
        text = '\n'.join(lines)
        message = f'\n{text}'

        if mock_mode:
            with outputs[i]:
                print(message)

        for receiver in clients:
            token = tok_tbl[receiver]
            if not mock_mode:
                line.notify_message(message, token)
            if not mock_mode:
                time.sleep(3)

def notify_with_checks():
    if period in ('Today', 'Yesterday'):
        if frequency not in ('Daily', 'Any'):
            print(f'The period is "{period}"')
            print('The frequency must be "Daily" or "Any"')
            return
    elif period in ('Recent 7 Days',):
        if frequency not in ('Weekly', 'Any'):
            print(f'The period is "{period}"')
            print('The frequency must be "Weekly" or "Any"')
            return
    elif period in ('Recent 2 Days',):
        if frequency not in ('2 Daily', 'Any'):
            print(f'The period is "{period}"')
            print('The frequency must be "2 Daily" or "Any"')
            return
    notify()

notify_with_checks()

### References
* [LINE Notify ::My page](https://notify-bot.line.me/my/)

#### Format of Config Files

subscriptions_Daily.yml:
```yaml
- [['Tesla & SpaceX; Vehicle'], [GroupA, GroupB]]
- [['Tech Titans'], [GroupA, GroupB]]
- [[Finance], [GroupA, GroupB, GroupC]]
- [[Taiwan], [GroupA, GroupB, GroupC]]
- [[Crypto], [GroupA]]
- [[IT, Science, '#AI', '#Robot'], [GroupA, GroupB, GroupC]]
```

subscriptions_Weekly.yml:
```yaml
- [[Technology], [GroupA, GroupB, GroupC]]
```

access_tokens.yml:
```yaml
GroupA: LINE_NOTIFY_ACCESS_TOKEN_OF_GROUP_A
GroupB: LINE_NOTIFY_ACCESS_TOKEN_OF_GROUP_B
GroupC: LINE_NOTIFY_ACCESS_TOKEN_OF_GROUP_C
```

### Test

In [5]:
#print(subscriptions)
#print(tok_tbl)
#display(output) # output from Step 1

In [6]:
#drive.flush_and_unmount()
#print('All changes made in this colab session should now be visible in Drive.')